In [14]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import pandas as pd
import time
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
processed_dataset = "/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/processed_datasets/2024-2-16_1915hours_8jun_300_win_300twin.csv"

In [3]:
df = pd.read_csv(processed_dataset)

/tmp/ipykernel_435494/1329464943.py:1: DtypeWarning: Columns (129,131,132,134) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(processed_dataset)


In [4]:
df2 = df[315:]
Y = df2["accident_label"]
X = df2.drop(["incident_edge","incident_start_time","incident_type","accident_id","accident_duration","incident_lane","accident_label"],axis=1)

In [5]:
X = X.drop(["rolling_travel_time_32_16"],axis=1)
X = X.fillna(-1)

In [6]:
#processing df

def standardize(col):
   return (col - col.mean()) / col.std()

X = X.apply(standardize)

In [7]:
print(X.isna().sum())

Unnamed: 0                        0
step                              0
rolling_junction_mean_speed_0     0
rolling_traffic_count_0           0
rolling_traffic_occupancy_0       0
rolling_junction_mean_speed_1     0
rolling_traffic_count_1           0
rolling_traffic_occupancy_1       0
rolling_junction_mean_speed_2     0
rolling_traffic_count_2           0
rolling_traffic_occupancy_2       0
rolling_junction_mean_speed_3     0
rolling_traffic_count_3           0
rolling_traffic_occupancy_3       0
rolling_junction_mean_speed_4     0
rolling_traffic_count_4           0
rolling_traffic_occupancy_4       0
rolling_junction_mean_speed_5     0
rolling_traffic_count_5           0
rolling_traffic_occupancy_5       0
rolling_junction_mean_speed_6     0
rolling_traffic_count_6           0
rolling_traffic_occupancy_6       0
rolling_junction_mean_speed_7     0
rolling_traffic_count_7           0
rolling_traffic_occupancy_7       0
rolling_junction_mean_speed_8     0
rolling_traffic_count_8     

In [10]:
int(Y.values[10])

0

In [12]:
class TrafficDataset(Dataset):
    def __init__(self, X, Y):
        # Convert the dataframes to tensors once, if they fit in memory
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.Y = torch.tensor(Y.values, dtype=torch.long)  # Use torch.long for classification indices
        
        self.num_files = len(self.X)
        
    def __len__(self):
        return self.num_files
    
    def __getitem__(self, index):
        # Directly access the pre-converted tensors
        row = self.X[index]
        label_data = self.Y[index]
        
        return row, label_data

In [13]:
print(f"Is CUDA available : {torch.cuda.is_available()}")
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

Is CUDA available : True


In [ ]:
class CNN_tabular(nn.Module):
    def __init__(self,input_features,output_features,num_classes):
        super(CNN_tabular, self).__init__()

        self.fc1 = nn.Linear(input_features, output_features)
        
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1) 
        # Optional: Define a pooling layer
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        
        # Calculate the size of the flattened features after convolutions and pooling
        self.flattened_size = 32 * (expanded_features // 2)  # Assuming one pooling layer
        
        # Fully connected layers for classification
        self.fc1 = nn.Linear(self.flattened_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        
        expanded = F.relu(self.fc1(x))
        
        x = x.unsqueeze(1)  # Add a channel dimension
        
        # Convolution layers with activation function
        x = F.relu(self.conv1(x))
        x = self.pool(x)  # Applying pooling
        x = F.relu(self.conv2(x))
        
        # Flatten the output for dense layers
        x = x.view(-1, self.flattened_size)
        
        # Dense layers for classification
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x),dim=1)
        
        return x

In [ ]:
# Train - Test splits

dataset = TrafficDataset(X,Y)

# Calculate the lengths of splits
total_size = len(dataset)
train_size = int(0.9 * total_size)
test_size = total_size - train_size

print(f"Train size is : {train_size}")
print(f"Test size is : {test_size}")

# Split the dataset
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for both datasets
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0) # num of workers = 0 in windows
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0) # Shuffle is usually False for testing/validation



In [ ]:
num_epochs  = 40

n_classes = 2

model = CNN_tabular()

loss_function = nn.CrossEntropyLoss(weight=weights)
# loss_function = FocalLoss(alpha=weights, gamma=3)
lr = 0.0001
optimizer = optim.Adam(model.parameters(), lr=lr)

model.to(device)
correct_predictions_per_class = torch.zeros(n_classes, device=device)
actual_per_class = torch.zeros(n_classes, device=device)

In [ ]:
print(f"Device used is {device}")
global_step = 0
training_loss_data = [] # Stores total training loss for each epoch
testing_loss_data = [] # Stores total testing loss for each epoch
epochs = []
training_time = 0 # Stores the total training time

In [ ]:
print(f"Training started for {num_epochs} epochs")

for epoch in range(num_epochs):
    
    epochs .append(epoch) 
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    start_time = time.time()
    total_predictions =0
    
    for i, (inputs, labels) in enumerate(train_dataloader):
        
        inputs, labels = inputs.to(device), labels.to(device) # Move data to the device
        optimizer.zero_grad()

        outputs = model(inputs)

        loss = loss_function(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()


        running_loss += loss.item()

        _, predicted = outputs.data
        _, labels_indices = labels

        if predicted >= 0.5:
            predicted =1
            
        
        
        predicted = predicted.to(labels_indices.device).long() 
        labels_indices = labels_indices.to(predicted.device).long()

        correct_predictions += (predicted == labels_indices).sum().item()
        total_predictions = total_predictions+32


        if i % 100 == 0:  # Log every 10 batches
            print(f"Epoch {epoch+1}, Batch {i+1}/{len(train_dataloader)}, Partial Loss: {running_loss/(i+1):.4f}, Correct Predictions: {correct_predictions}/{total_predictions}")
            

        global_step+=1

    
    epoch_time = time.time() - start_time
    epoch_loss = running_loss / len(train_dataloader)
    training_loss_data.append(epoch_loss)
    epoch_accuracy = (correct_predictions / total_samples) * 100
    print(f"Epoch {epoch+1} completed in {epoch_time:.2f} seconds.")
    # print(f"Before Learning rate {before_lr}, After Learning rate {after_lr}")
    
    print(f"Epoch {epoch+1} Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
    # Testing phase ---------------------------------------------------------------------------
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # No gradients needed for validation
        correct_test_predictions = 0
        total_test_samples = 0
        test_loss = 0
        for test_inputs, test_labels in test_dataloader:
            test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)  
            outputs = model(test_inputs)
            loss = loss_function(outputs, test_labels)
            test_loss += loss.item()
            _, predicted = outputs.data
            if predicted >= 0.5:
                predicted = 1
            total_test_samples += test_labels.size(0)
            _, labels_indices = test_labels

            correct_test_predictions += (predicted == labels_indices).sum().item()
    testing_loss_data.append(test_loss/len(test_dataloader))
    test_accuracy = (correct_test_predictions / total_test_samples) * 100
    print(f"Epoch {epoch+1} Test Accuracy: {test_accuracy:.2f}% Test Loss {test_loss/len(test_dataloader)}\n")



In [ ]:
# Training loss vs epochs
plt.plot(epochs,training_loss_data)
plt.plot(epochs,testing_loss_data)
plt.title("Training loss vs epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

     

In [16]:
import os



print(files)

['/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/processed_datasets/2024-2-16_1915hours_3jun_300_win_300twin.csv', '/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/processed_datasets/2024-2-16_1915hours_3jun_600_win_300twin.csv', '/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/processed_datasets/2024-2-16_1915hours_3jun_600_win_600twin.csv', '/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/processed_datasets/2024-2-16_1915hours_3jun_900_win_300twin.csv', '/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/processed_datasets/2024-2-16_1915hours_4jun_300_win_300twin.csv', '/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/processed_datasets/2024-2-16_1915hours_4jun_600_win_300twin.csv', '/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/processed_datasets/2024-2-16_1915hours_4jun_600_win_600twin.csv', '/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/processed_datasets/2024-2-16_1915